In [3]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
import nltk
import pandas as pd
import numpy as np
import spacy

sp = spacy.load('en_core_web_sm')

In [22]:
file = pd.read_csv(r"joined_train.txt", sep='|' , encoding='latin-1')
file_validation = pd.read_csv(r"joined_validation.txt", sep='|' , encoding='latin-1')

In [ ]:
for entity in sentence5.ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

In [ ]:
sp("alfons kemper")

In [ ]:
apples = sp("I like apples")
oranges = sp("I like oranges")
apples_oranges = apples.similarity(oranges)
oranges_apples = oranges.similarity(apples)

In [ ]:
apples_oranges

In [ ]:
#for token in sentence5:
#    print (token, token.tag_, token.pos_, spacy.explain(token.tag_))

In [5]:
def get_tokens_features(sentence_):
    sentence_sp = sp(sentence_)  
    list_sentence = list(sentence_sp)
    list_tokens_str = [str(i) for i in list_sentence]
    return set(list_tokens_str)

In [24]:
def prepare_features_fuzzy(input_df):
    
    joined_dataframe_with_references = input_df.copy()
    
    list1 = list(joined_dataframe_with_references['desc_without_puct.x'])
    list2 = list(joined_dataframe_with_references['desc_without_puct.y'])
    
    joined_dataframe_with_references["fuzzy_similarity"] = [fuzz.ratio(list1[i], list2[i]) for i in range(len(list1))]
    joined_dataframe_with_references["fuzzy_similarity_sorted_tokens"] = [fuzz.token_sort_ratio(list1[i], list2[i]) for i in range(len(list1))]
    joined_dataframe_with_references["fuzzy_similarity_partial_ratio"] = [fuzz.partial_ratio(list1[i], list2[i]) for i in range(len(list1))]
    
    
    joined_dataframe_with_references["set_tokens_x"] = [get_tokens_features(i) for i in list1]
    joined_dataframe_with_references["set_tokens_y"] = [get_tokens_features(i) for i in list2]  
    joined_dataframe_with_references["common_tokens"] = [list(joined_dataframe_with_references["set_tokens_x"])[i].intersection(list(joined_dataframe_with_references["set_tokens_y"])[i]) for i in range(joined_dataframe_with_references.shape[0])]
    
    joined_dataframe_with_references["len_tokens_x"] = [len(i) for i in list(joined_dataframe_with_references["set_tokens_x"])]
    joined_dataframe_with_references["len_tokens_y"] = [len(i) for i in list(joined_dataframe_with_references["set_tokens_y"])]
    joined_dataframe_with_references["len_common_tokens"] = [len(i) for i in list(joined_dataframe_with_references["common_tokens"])]
    
    joined_dataframe_with_references["common_to_smaller_ratio"] = joined_dataframe_with_references["len_common_tokens"]/joined_dataframe_with_references[["len_tokens_x", "len_tokens_y"]].min(axis=1)
    joined_dataframe_with_references["common_to_longer_ratio"] = joined_dataframe_with_references["len_common_tokens"]/joined_dataframe_with_references[["len_tokens_x", "len_tokens_y"]].max(axis=1)
    
    return joined_dataframe_with_references

In [25]:
prepared_file = prepare_features_fuzzy(file)

In [28]:
prepared_file_validation = prepare_features_fuzzy(file_validation)

In [31]:
data_to_model = prepared_file[["label"
                               , "ltable_id"
                               , "rtable_id"
                               , "possible_year.x"
                               , "possible_year.y"
                               , 'short_words_removed.x'
                               , 'short_words_removed.y'
                               , "fuzzy_similarity"
                               , "fuzzy_similarity_sorted_tokens"
                               , "fuzzy_similarity_partial_ratio"
                               , 'len_tokens_x'
                               , 'len_tokens_y'
                               , 'len_common_tokens'
                               , 'common_to_smaller_ratio'
                               , 'common_to_longer_ratio']]

In [32]:
data_to_predict = prepared_file_validation[["label"
                               , "ltable_id"
                               , "rtable_id"
                               , "possible_year.x"
                               , "possible_year.y"
                               , 'short_words_removed.x'
                               , 'short_words_removed.y'
                               , "fuzzy_similarity"
                               , "fuzzy_similarity_sorted_tokens"
                               , "fuzzy_similarity_partial_ratio"
                               , 'len_tokens_x'
                               , 'len_tokens_y'
                               , 'len_common_tokens'
                               , 'common_to_smaller_ratio'
                               , 'common_to_longer_ratio']]

In [33]:
data_to_model.to_csv(r'to_model.txt', header=True, index=None, sep='|', mode='w')

In [34]:
data_to_predict.to_csv(r'to_predict.txt', header=True, index=None, sep='|', mode='w')